# Marker gene database maker
The purpose of this jupyter notebook is to run through a workflow of creating a blast database containing protein sequences of a given gene from a wide range of taxonomic groups that can be used to validate newly submitted sequences against. 

Broadly, this process involves the following steps: 

1. Starting with an Entrez query for the Gene database, download sequences and metadata for genes, transcripts and proteins using NCBI Datasets
2. Parse the data archive from step 1 to tabulate names and symbols for review
3. Parse the data archive from step 1 to tabulate variability in the sequence lengths for review 
4. Given a set of taxonomic group identifiers, tabulate the number of sequences for each group that are present in the data archive
5. Extract sequences from each taxonomic node and generate all-vs-all BLAST alignments 
6. Review the BLAST tabular output to make a list of accessions that are outliers or incorrect that need to be removed from the final BLAST database 
7. Generate a final BLAST database that can be used with VADR and other tools for validating newly submitted sequences.

## Download data

Sequence and metadata are downloaded using NCBI Datasets using an Entrez query provided by the user. 

In [100]:
## specify Entrez query and output filename
entrez_query = 'mammalia [ORGN] AND cytb [GENE] AND source mitochondrion [PROP] NOT rnatype mrna [PROP] NOT srcdb pdb [PROP] NOT uncultured NOT unverified'
email = 'email@example.com'
output_file = 'ncbi_dataset.zip'

In [101]:
import scripts.obtain_gene_datasets as dl

gene_ids_file = 'gene_ids.txt'
dl.populate_gene_ids_file(entrez_query, email, gene_ids_file)
json_data = dl.format_file_data_into_json(gene_ids_file)
dl.obtain_gene_datasets(json_data, output_file)

## Unzip Datasets archive

In [32]:
!unzip {output_file}

Archive:  ncbi_dataset.zip
  inflating: ncbi_dataset/bagit.txt  
  inflating: README.md               
  inflating: ncbi_dataset/data/gene.fna
  inflating: ncbi_dataset/data/rna.fna
  inflating: ncbi_dataset/data/protein.faa
  inflating: ncbi_dataset/data/data_report.yaml
  inflating: ncbi_dataset/data/data_table.tsv  
  inflating: ncbi_dataset/data/dataset_catalog.json  
  inflating: ncbi_dataset/fetch.txt  
  inflating: ncbi_dataset/bag-info.txt  
  inflating: ncbi_dataset/manifest-md5.txt  
  inflating: ncbi_dataset/tagmanifest-md5.txt  


## Interlude

The `data_table.tsv` file has a lot of rows that use comma as delimiter instead of tab. This bug in Datasets will be fixed in the future but for now, the following steps are needed to prepare `data_table.tsv` for downstream steps. 

In [83]:
bdbag = 'ncbi_dataset/'
data_table = bdbag + 'data/data_table.tsv'

In [70]:
%%bash -s {data_table}

data_table=$1

# replace all cases of ', ' (comma + space) with a semi-colon
# they are all legitimate uses of comma
sed -i '/\t/! s/, /%2C/g' ${data_table}

# if a tab character is not found in line, replace commas with tab
sed -i '/\t/! s/,/\t/g' ${data_table}

# change all instances of 3 consecutive semi-colons to comma
sed -i 's/%2C/, /g' ${data_table}

# check to make sure all rows have 18 fields only
# if everything worked, there should not be any output for this command 
awk 'BEGIN{FS="\t";OFS="\t"}(NF!=18){print NF,$0}' ${data_table}

## Tabulate unique names

In [102]:
gene_names = 'gene_names.tsv'

In [103]:
%%bash -s {data_table} {gene_names}

data_table=$1
gene_names=$2

python3 scripts/unique.py ${data_table} > ${gene_names}

In [104]:
import pandas as pd 

df = pd.read_csv(gene_names, sep='\t', header=None, names=['Gene Name', 'Count', 'Gene IDs'])
display(df.sort_values(by=['Count'], ascending=False))

,Gene Name,Count,Gene IDs
1,CYTB,9353,NaN
2,MT-CYB,2,"140512,4519"
3,MT-CYTB,2,"17711,26192"
0,"NAME: CYTOCHROMEB, MITOCHONDRIAL",1,41954867


## Identify outliers based on protein size

In [105]:
data_table_df = pd.read_csv(data_table, sep='\t', index_col=1)
data_table_df.head()

,gene_id,description,scientific_name,common_name,tax_id,genomic_range,orientation,location,gene_type,transcript_accession,transcript_name,transcript_length,transcript_cds_coords,protein_accession,isoform_name,protein_length,protein_name
gene_symbol,,,,,,,,,,,,,,,,,
CYTB,10002870,cytochrome b,Entoria okinawaensis,Okinawa walking stick,590984,NC_014694.1:10236-11367,+,NaN,PROTEIN_CODING,NC_014694.1,cytochrome b,377.0,NaN,YP_004021616.1,NaN,377.0,cytochrome b [Entoria okinawaensis]
CYTB,10007423,cytochrome b,Protopterus aethiopicus,marbled lungfish,7886,NC_014764.2:14186-15325,+,NaN,PROTEIN_CODING,NC_014764.2,cytochrome b,379.0,NaN,YP_004061423.1,NaN,379.0,cytochrome b [Protopterus aethiopicus]
CYTB,10020612,cytochrome b,Taenia taeniaeformis,NaN,6205,NC_014768.1:3137-4183,+,NaN,PROTEIN_CODING,NC_014768.1,cytochrome b,348.0,NaN,YP_004062130.1,NaN,348.0,cytochrome b [Taenia taeniaeformis]
CYTB,10020636,cytochrome b,Cuora amboinensis,Amboina box turtle,74909,NC_014769.1:14242-15381,+,NaN,PROTEIN_CODING,NC_014769.1,cytochrome b,379.0,NaN,YP_004062152.1,NaN,379.0,cytochrome b [Cuora amboinensis]
CYTB,10020650,cytochrome b,Panthera tigris amoyensis,Amoy tiger,253258,NC_014770.1:15113-16252,+,NaN,PROTEIN_CODING,NC_014770.1,cytochrome b,379.0,NaN,YP_004062165.1,NaN,379.0,cytochrome b [Panthera tigris amoyensis]


In [106]:
data_table_df[['transcript_length', 'protein_length']].describe()

,transcript_length,protein_length
count,9315.000000,9315.000000
mean,378.693398,378.693398
std,5.964827,5.964827
min,249.000000,249.000000
25%,378.000000,378.000000
50%,380.000000,380.000000
75%,380.000000,380.000000
max,530.000000,530.000000


In [115]:
## filter sequences that are either too long or too short

min_len = 350
max_len = 400

rightlength = data_table_df.loc[(data_table_df['protein_length'] > min_len) & (data_table_df['protein_length'] < max_len)]
rightlength.to_csv(data_table, sep='\t')
rightlength.head()


,gene_id,description,scientific_name,common_name,tax_id,genomic_range,orientation,location,gene_type,transcript_accession,transcript_name,transcript_length,transcript_cds_coords,protein_accession,isoform_name,protein_length,protein_name
gene_symbol,,,,,,,,,,,,,,,,,
CYTB,10002870,cytochrome b,Entoria okinawaensis,Okinawa walking stick,590984,NC_014694.1:10236-11367,+,NaN,PROTEIN_CODING,NC_014694.1,cytochrome b,377.0,NaN,YP_004021616.1,NaN,377.0,cytochrome b [Entoria okinawaensis]
CYTB,10007423,cytochrome b,Protopterus aethiopicus,marbled lungfish,7886,NC_014764.2:14186-15325,+,NaN,PROTEIN_CODING,NC_014764.2,cytochrome b,379.0,NaN,YP_004061423.1,NaN,379.0,cytochrome b [Protopterus aethiopicus]
CYTB,10020636,cytochrome b,Cuora amboinensis,Amboina box turtle,74909,NC_014769.1:14242-15381,+,NaN,PROTEIN_CODING,NC_014769.1,cytochrome b,379.0,NaN,YP_004062152.1,NaN,379.0,cytochrome b [Cuora amboinensis]
CYTB,10020650,cytochrome b,Panthera tigris amoyensis,Amoy tiger,253258,NC_014770.1:15113-16252,+,NaN,PROTEIN_CODING,NC_014770.1,cytochrome b,379.0,NaN,YP_004062165.1,NaN,379.0,cytochrome b [Panthera tigris amoyensis]
CYTB,10043687,cytochrome b,Grapholita molesta,oriental fruit moth,192188,NC_014806.1:10492-11637,+,NaN,PROTEIN_CODING,NC_014806.1,cytochrome b,381.0,NaN,YP_004072439.1,NaN,381.0,cytochrome b [Grapholita molesta]


## Extract sequences from specific taxonomic group(s) for further analysis

Analyzing all of the sequences using all-vs-all BLAST is time-consuming. In this step, we will group the sequences into broad taxonomic groups for further analysis. 

In [116]:
acclist_for_blast = 'acclist_for_blast.tsv'
taxids = !cut -f2 example_data/tax_nodes.tsv | head -n 3 | paste -s -d ','
print(taxids)

['6073,10226,6042']


In [118]:
!python3 scripts/seqids_by_taxa.py --bdbag {bdbag} --taxids {taxids[0]} --output {acclist_for_blast} --email {email}

6073	184
10226	3
6042	37


## Run all-vs-all BLAST

In [119]:
!scripts/blast_all.sh -b {bdbag} -a {acclist_for_blast} -t 6

Number of taxids in acclist_for_blast.tsv: 3
Processing 10226
Fri Jul 10 18:18:41 UTC 2020 Filtering protein fasta...
Fri Jul 10 18:18:41 UTC 2020 Create a BLAST database...


Building a new DB, current time: 07/10/2020 18:18:41
New DB name:   /home/kodalivk/Marker-Gene-Validator/10226_blastdb
New DB title:  10226_input.fa
Sequence type: Protein
Deleted existing Protein BLAST database named /home/kodalivk/Marker-Gene-Validator/10226_blastdb
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 3 sequences in 0.00916195 seconds.


Fri Jul 10 18:18:41 UTC 2020 Running all-vs-all blast...
Fri Jul 10 18:18:41 UTC 2020 Generating blast tabular output...
Fri Jul 10 18:18:41 UTC 2020 Generating blast seq-align asn...
Processing 6042
Fri Jul 10 18:18:41 UTC 2020 Filtering protein fasta...
Fri Jul 10 18:18:41 UTC 2020 Create a BLAST database...


Building a new DB, current time: 07/10/2020 18:18:41
New DB name:   /home/kodalivk/Marker-Gene-Validator/6042_blastdb
New DB